In [1]:
%load_ext autoreload

In [2]:
import sys
sys.version

'3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]'

In [3]:
import functools
import time
import logging
from multiprocessing import shared_memory as shm
import multiprocessing
import json
from io import BytesIO

log = logging.getLogger(__name__)

import sparse
import scipy.sparse
import numpy as np
import cloudpickle

from libertem.masks import to_dense, to_sparse, is_sparse
from libertem.common import Slice

In [4]:
import shmem

In [5]:
ref_queue = multiprocessing.JoinableQueue()

In [6]:
%autoreload
ref_p = multiprocessing.Process(target=shmem.keep_a_reference, args=(ref_queue, ))
print(f"starting {ref_p}")
ref_p.start()
print(ref_p)

starting <Process name='Process-1' parent=17092 initial>
<Process name='Process-1' pid=22408 parent=17092 started>


In [7]:
ref_queue.put('hurz')

In [8]:
ref_p

<Process name='Process-1' pid=22408 parent=17092 started>

In [14]:
f = functools.partial(sparse.eye, 13)

In [10]:
%autoreload
m = shmem.load_or_create(f, ref_queue)

stored 0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c


In [11]:
%autoreload
shmem.load_or_create(f, ref_queue)

[WinError 183] File exists: '0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c'
loading 0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c...


(<COO: shape=(12, 12), dtype=float64, nnz=12, fill_value=0.0>,
 [SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c_meta', size=4096),
  SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c_coords', size=4096),
  SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c_data', size=4096),
  SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c', size=4096)])

In [12]:
shmem.load_or_create(f, ref_queue)

[WinError 183] File exists: '0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c'
loading 0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c...


(<COO: shape=(12, 12), dtype=float64, nnz=12, fill_value=0.0>,
 [SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c_meta', size=4096),
  SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c_coords', size=4096),
  SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c_data', size=4096),
  SharedMemory('0_ffb6a9f3e5c80e8ed2bad9976551b6c54caed58515e27ac2feb2002bf30a59331e93c90e0aa706820490a6d0e3ba28461e81a009776a5fc809043ef310c2859c', size=4096)])

In [15]:
if __name__ == '__main__':
    processes = []
    for i in range(5):
        p = multiprocessing.Process(target=shmem.load_or_create, args=(f, ref_queue))
        print(f"starting {p}")
        p.start()
        print(p)
        processes.append(p)
    for p in processes:
        print(f"joining {p}...")
        p.join()
        print(f"Done {p}.")

starting <Process name='Process-7' parent=17092 initial>
<Process name='Process-7' pid=20272 parent=17092 started>
starting <Process name='Process-8' parent=17092 initial>
<Process name='Process-8' pid=17652 parent=17092 started>
starting <Process name='Process-9' parent=17092 initial>
<Process name='Process-9' pid=6692 parent=17092 started>
starting <Process name='Process-10' parent=17092 initial>
<Process name='Process-10' pid=14920 parent=17092 started>
starting <Process name='Process-11' parent=17092 initial>
<Process name='Process-11' pid=9808 parent=17092 started>
joining <Process name='Process-7' pid=20272 parent=17092 started>...
Done <Process name='Process-7' pid=20272 parent=17092 stopped exitcode=0>.
joining <Process name='Process-8' pid=17652 parent=17092 started>...
Done <Process name='Process-8' pid=17652 parent=17092 stopped exitcode=0>.
joining <Process name='Process-9' pid=6692 parent=17092 stopped exitcode=0>...
Done <Process name='Process-9' pid=6692 parent=17092 sto